In [70]:
import os
import toml

In [71]:
# Load the TOML file
with open('config.toml', 'r') as f:
    config = toml.load(f)

# Set the environment variables
for key, value in config.items():
    os.environ[key] = str(value)


In [72]:
import ibis
con = ibis.duckdb.connect('md:Eunomia')

In [73]:
print(con.list_tables())

['CARE_SITE', 'CDM_SOURCE', 'COHORT', 'COHORT_ATTRIBUTE', 'CONCEPT', 'CONCEPT_ANCESTOR', 'CONCEPT_CLASS', 'CONCEPT_RELATIONSHIP', 'CONCEPT_SYNONYM', 'CONDITION_ERA', 'CONDITION_OCCURRENCE', 'COST', 'DEATH', 'DEVICE_EXPOSURE', 'DOMAIN', 'DOSE_ERA', 'DRUG_ERA', 'DRUG_EXPOSURE', 'DRUG_STRENGTH', 'FACT_RELATIONSHIP', 'LOCATION', 'MEASUREMENT', 'METADATA', 'NOTE', 'NOTE_NLP', 'OBSERVATION', 'OBSERVATION_PERIOD', 'PAYER_PLAN_PERIOD', 'PERSON', 'PROCEDURE_OCCURRENCE', 'PROVIDER', 'RELATIONSHIP', 'SOURCE_TO_CONCEPT_MAP', 'SPECIMEN', 'VISIT_DETAIL', 'VISIT_OCCURRENCE', 'VOCABULARY', 'ambient_air_quality', 'coxibVsNonselVsGiBleed', 'hacker_news', 'outcomes', 'rideshare', 'service_requests', 'taxi']


In [74]:
import re
from pydantic import BaseModel, confloat
from datetime import datetime, date
import ibis

con = ibis.duckdb.connect('md:Eunomia')

table_name = 'PROCEDURE_OCCURRENCE'

table = con.table(f'{table_name}')

def ibis_schema_to_pydantic(schema_str: str, class_name: str) -> str:
    # Parse the schema string into a dictionary
    schema_dict = {}
    for match in re.findall(r'(\w+)\s+(\w+)', schema_str):
        schema_dict[match[0]] = match[1]

    # Generate the Pydantic fields
    pydantic_fields = []
    for name, dtype in schema_dict.items():
        if name.endswith('_DATETIME'):
            pydantic_type = 'datetime'
            default_value = 'None'
        elif name.endswith('_DATE'):
            pydantic_type = 'datetime'
            default_value = 'None'
        elif name.endswith('_ID'):
            pydantic_type = 'int'
            default_value = 'None'
        else:
            pydantic_type = 'int' if dtype in ('int32', 'int64', 'float64') else 'str'
            default_value = 'None' if pydantic_type == 'Optional[str]' else '...'
        
        # Handle 'None' string as default value for datetime fields
        if pydantic_type == 'datetime' and default_value == 'None':
            default_value = None
        
        pydantic_fields.append(f'{name.lower()}: Optional[{pydantic_type}] = Field({default_value}, alias=\'{name}\')')

    # Generate the Pydantic model with the derived class name
    class_name = ''.join(word.capitalize() for word in re.split(r'[^a-zA-Z0-9_]', class_name))
    class_name = class_name[:1].upper() + class_name[1:]
    class_name = re.sub(r'_(\w)', lambda m: m.group(1).upper(), class_name)
    return f'class {class_name}(BaseModel):\n    ' + '\n    '.join(pydantic_fields)

schema_str = str(table.schema())
# Derive the class name from the table name
class_name = ''.join(word.capitalize() for word in re.split(r'[^a-zA-Z0-9_]', table_name))
# Test the function
print(ibis_schema_to_pydantic(schema_str, class_name))

class ProcedureOccurrence(BaseModel):
    procedure_occurrence_id: Optional[int] = Field(None, alias='PROCEDURE_OCCURRENCE_ID')
    person_id: Optional[int] = Field(None, alias='PERSON_ID')
    procedure_concept_id: Optional[int] = Field(None, alias='PROCEDURE_CONCEPT_ID')
    procedure_date: Optional[datetime] = Field(None, alias='PROCEDURE_DATE')
    procedure_datetime: Optional[datetime] = Field(None, alias='PROCEDURE_DATETIME')
    procedure_type_concept_id: Optional[int] = Field(None, alias='PROCEDURE_TYPE_CONCEPT_ID')
    modifier_concept_id: Optional[int] = Field(None, alias='MODIFIER_CONCEPT_ID')
    quantity: Optional[int] = Field(..., alias='QUANTITY')
    provider_id: Optional[int] = Field(None, alias='PROVIDER_ID')
    visit_occurrence_id: Optional[int] = Field(None, alias='VISIT_OCCURRENCE_ID')
    visit_detail_id: Optional[int] = Field(None, alias='VISIT_DETAIL_ID')
    procedure_source_value: Optional[str] = Field(..., alias='PROCEDURE_SOURCE_VALUE')
    procedure_source

In [78]:
from pydantic import BaseModel, Field, ValidationError
from typing import Optional, Type
from datetime import date, datetime
import pandas as pd
import numpy as np

def validate_dataframe(df: pd.DataFrame, model: Type[BaseModel]) -> pd.DataFrame:
    for index, row in df.iterrows():
        try:
            # Convert values to None if they are NaN or None
            row_dict = {k: v if not pd.isna(v) and v is not None else None for k, v in row.to_dict().items()}
            model(**row_dict)
        except ValidationError as e:
            print(f"Validation error for row {index}: {e}")
    return df

class ProcedureOccurrence(BaseModel):
    procedure_occurrence_id: Optional[int] = Field(None, alias='PROCEDURE_OCCURRENCE_ID')
    person_id: Optional[int] = Field(None, alias='PERSON_ID')
    procedure_concept_id: Optional[int] = Field(None, alias='PROCEDURE_CONCEPT_ID')
    procedure_date: Optional[datetime] = Field(None, alias='PROCEDURE_DATE')
    procedure_datetime: Optional[datetime] = Field(None, alias='PROCEDURE_DATETIME')
    procedure_type_concept_id: Optional[int] = Field(None, alias='PROCEDURE_TYPE_CONCEPT_ID')
    modifier_concept_id: Optional[int] = Field(None, alias='MODIFIER_CONCEPT_ID')
    quantity: Optional[int] = Field(..., alias='QUANTITY')
    provider_id: Optional[int] = Field(None, alias='PROVIDER_ID')
    visit_occurrence_id: Optional[int] = Field(None, alias='VISIT_OCCURRENCE_ID')
    visit_detail_id: Optional[int] = Field(None, alias='VISIT_DETAIL_ID')
    procedure_source_value: Optional[str] = Field(..., alias='PROCEDURE_SOURCE_VALUE')
    procedure_source_concept_id: Optional[int] = Field(None, alias='PROCEDURE_SOURCE_CONCEPT_ID')
    modifier_source_value: Optional[int] = Field(..., alias='MODIFIER_SOURCE_VALUE')

In [79]:
df = table.execute()

validate_dataframe(df, ProcedureOccurrence)

,PROCEDURE_OCCURRENCE_ID,PERSON_ID,PROCEDURE_CONCEPT_ID,PROCEDURE_DATE,PROCEDURE_DATETIME,PROCEDURE_TYPE_CONCEPT_ID,MODIFIER_CONCEPT_ID,QUANTITY,PROVIDER_ID,VISIT_OCCURRENCE_ID,VISIT_DETAIL_ID,PROCEDURE_SOURCE_VALUE,PROCEDURE_SOURCE_CONCEPT_ID,MODIFIER_SOURCE_VALUE
0,3554.0,343.0,4107731.0,6.969024e+08,6.969024e+08,38000275.0,0.0,None,None,22951.0,0.0,180256009,4107731.0,None
1,3741.0,357.0,4107731.0,6.585408e+08,6.585408e+08,38000275.0,0.0,None,None,23670.0,0.0,180256009,4107731.0,None
2,3928.0,399.0,4107731.0,6.842880e+08,6.842880e+08,38000275.0,0.0,None,None,26205.0,0.0,180256009,4107731.0,None
3,4115.0,406.0,4107731.0,6.348672e+08,6.348672e+08,38000275.0,0.0,None,None,26759.0,0.0,180256009,4107731.0,None
4,4302.0,411.0,4107731.0,6.950880e+08,6.950880e+08,38000275.0,0.0,None,None,27401.0,0.0,180256009,4107731.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37404,53101.0,4613.0,4275113.0,8.876736e+08,8.876736e+08,38000275.0,0.0,None,None,307277.0,0.0,65200003,4275113.0,None
37405,53102.0,4613.0,4125906.0,3.373920e+08,3.373920e+08,38000275.0,0.0,None,None,307231.0,0.0,288086009,4125906.0,None
37406,53103.0,4613.0,4186930.0,1.459037e+09,1.459037e+09,38000275.0,0.0,None,None,307298.0,0.0,46706006,4186930.0,None
37407,53104.0,4613.0,4186930.0,8.276256e+08,8.276256e+08,38000275.0,0.0,None,None,307250.0,0.0,46706006,4186930.0,None
